## Data Prepare & Labeled

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd

In [3]:
sample_df = pd.read_excel('dataset/sample_1000.xlsx')
sample_df['category'] = sample_df['情感倾向'].apply(lambda x: 'O' if x == '其他' else x)

In [5]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer,pipeline

model = AutoModelForSequenceClassification.from_pretrained('uer/roberta-base-finetuned-dianping-chinese')
tokenizer = AutoTokenizer.from_pretrained('uer/roberta-base-finetuned-dianping-chinese')
text_classification = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [13]:
text_classification(sample_df.iloc[0]['content'][0:100])

[{'label': 'positive (stars 4 and 5)', 'score': 0.9491583108901978}]